# 베스트셀러 다중 카테고리 분석 (Category 1, 2, 3)
이 노트북은 도서에 부여된 최대 3개의 카테고리를 통합하여 도서의 성격을 더 깊이 있게 분석합니다.

In [ ]:
import os
import pandas as pd
import plotly.express as px
from dotenv import load_dotenv
from supabase import create_client

load_dotenv()
supabase = create_client(os.getenv('SUPABASE_URL'), os.getenv('SUPABASE_KEY'))

# 데이터 로드
bestsellers_res = supabase.table('bestsellers').select('*').execute()
df_bestsellers = pd.DataFrame(bestsellers_res.data)

books_res = supabase.table('books').select('product_code, title, category_1, category_2, category_3').execute()
df_books = pd.DataFrame(books_res.data)

# 데이터 병합
df = df_bestsellers.merge(df_books, on='product_code', how='left')

# None 값을 '미분류'로 채우기 (시각화 시 명확성 위해)
df['category_1'] = df['category_1'].fillna('미분류')
df['category_2'] = df['category_2'].fillna('None')
df['category_3'] = df['category_3'].fillna('None')

## 1. 계층적 카테고리 분석 (Sunburst Chart)
Category 1에서 3으로 이어지는 계층 구조를 시각화합니다. 중앙에서 바깥으로 갈수록 하위 카테고리로 매핑됩니다.

In [ ]:
fig1 = px.sunburst(df, path=['category_1', 'category_2', 'category_3'], 
                  title='베스트셀러 카테고리 계층 구조 (Sunburst)',
                  color='category_1')
fig1.show()

## 2. 통합 카테고리 영향력 (전체 카테고리 빈도)
Category 1, 2, 3 구분 없이 가장 많이 언급된 키워드(카테고리)를 추출합니다.

In [ ]:
# 모든 카테고리 열을 하나로 합침
all_cats = pd.concat([df['category_1'], df['category_2'], df['category_3']])
cat_counts = all_cats[all_cats != 'None'].value_counts().reset_index()
cat_counts.columns = ['category', 'frequency']

fig2 = px.bar(cat_counts.head(20), x='frequency', y='category', orientation='h',
             title='전체 베스트셀러 통합 카테고리 키워드 TOP 20',
             color='frequency', color_continuous_scale='Viridis')
fig2.update_layout(yaxis={'categoryorder':'total ascending'})
fig2.show()

## 3. 다중 카테고리 비중 (Icicle Chart)
전체 비중을 사각형 블록으로 보여주어 계층 간의 크기 비교가 더 용이합니다.

In [ ]:
fig3 = px.icicle(df, path=['category_1', 'category_2', 'category_3'], 
                title='베스트셀러 카테고리 계층 구조 (Icicle)',
                color='category_1')
fig3.show()